# JVM 基础
1. java内存分配是一段比较连续的堆空间，是有规律和顺序的，通过移动堆指针来分配新的空间，因此效率比c++可能都要快，c++的堆空间就是一块大的空间，需要处理销毁和重新利用等问题
2. 垃圾回收的方法：
    * 引用计数：对每个被引用的对象保持一个简单的使用计数，当计数等于0时就可以将该对象空间回收，缺点就是需要持续更新计数，并且遍历那些对象的引用计数变成0了，而且在出现循环引用时，很难排除问题
    * 寻找“活”的对象，从堆栈和静态区域开始遍历所有的引用，当找到“活”的引用后，从它的基础上继续遍历引用，直到把所有引用都遍历一遍（java虚拟机利用的是这种思想）
3. java垃圾回收机制，不同jvm实现方法不同，基本是以下两种模式的组合形成自适应的回收算法：
    * 停止-复制：先停止程序运行（不属于后台回收模式），遍历完“活”引用后，将引用复制到一个新的堆空间中去（生成新的连续的堆空间），并修正堆指针的指向，同时把老的堆空间全部释放，缺点是非常的耗性能，需要将程序暂停，然后在两个堆中反复的复制数据，某些jvm避免在两个堆中操作，会在现有的堆中开辟一个很大的堆内存，然后在本堆中完成复制操作，其次当只有少量垃圾产生时，这种模式还是会把整个堆都复制一遍，及其消耗性能，所以可以结合标记-清扫模式
    * 标记-清扫：先停止程序运行（不属于后台回收模式），遍历“活引用”，遍历的同时会给引用加标记，遍历完后如果没有加标记的引用则是无效的，会被清除，但是会产生不连续的堆空间
4. class文件组成结构
    * class文件头，指明文件类型
    * 常量池，UTF8常量类型、FieldRef/MethodRef常量类型、Class常量类型、NamedAndType常量类型
    * 类信息，访问控制、名称、类型
    * Field和Method定义
    * 类属性描述
5. jvm工具集，详情参考《深入理解java虚拟机》第97页，需要实践
    * 命令行工具：
        * jps：虚拟机进程状况工具
        * jstat：虚拟机统计信息监视工具
        * jinfo：java配置信息工具
        * jmap：java内存映射工具
        * jhat：虚拟机堆转储快照分析工具
        * jstack：java堆栈跟踪工具
    * 可视化工具：
        * JConsole：java监视与管理控制台
        * VisualVM：多合一故障处理工具

****

# JVM参数
* 详细GC配置参数《JavaWeb深入分析技术内幕》第243页
* JVM主要参数列表参考《深入理解java虚拟机》第393页
* JVM参数设置三种方式：<img src='../images/javase/jvm参数设置三种方式.png' width='300px'>
* 自定义存储空间：
    * -Xms：指定堆初始大小
    * -Xmx：指定堆最大值
    * -Xmn：指定Young区域大小
    * -Xss：指定栈大小
    * -XX:PermSize：指定永久代初始大小
    * -XX:MaxPermSize：指定永久代最大值
* GC相关设定
    * XX:+DisableExplicitGC：调整System.gc()
    * -XX:+PrintGCDetails：打印GC日志
    * -XX:+UseSerialGC：指定Serial Collector垃圾回收算法
    * -XX:PretenureSizeThreshold：指定Eden区触发minor GC阈值
    * -XX:HandlePromotionFailure：Young区往Old区转移时是否触发full GC标志
    * -XX:MaxTenuringThreshold：控制minor GC多少次才将对象从Eden区移到Old区：
    * -XX:+UseParNewGC：指定ParNewGC垃圾回收算法
    * -XX:+UseParallelGC： 指定ParallelGC垃圾回收算法
    * -XX:+UseParallelOldGC：指定ParallelOldGC垃圾回收算法
    * -XX:ParallelGCThreads：指定ParallelGC垃圾回收线程数量
    * -XX:SurvivorRatio：指定Eden区与Servivor区的比例
    * -XX:+UseConcMarkSweepGC：指定CMS Collector垃圾回收算法
    * -XX:ParallelCMSThreads：指定CMSParallelGC垃圾回收线程数量
    * -XX:CMSInitiatingOccupancyFraction：指定Old区使用率触发GC
    * -XX:+HeapDumpOnOutOfMemoryError：保存OutOfMemoryError时的快照
    * -XX:MaxDirectMemorySize，不指定与-Xmx保持一致：指定直接内存区大小
    * -verbose:class、-XX:+TraceClassLoading、-XX:+TraceClassUnLoading：查看类加载卸载信息
    * 详细列表：<img src='../images/javase/GC参数.png' width='400px'>
* 即时编译器设置
    * -client：强制指定即时编译器使用client模式
    * -server：强制指定即时编译器使用server模式
    * -Xint：强制即时编译器使用解释模式
    * -Xcomp：强制即时编译器使用编译模式
    * -XX:CompileThreshold：指定调用次数触发即时编译器工作的阈值
    * -XX:-UseCounterDecay：关闭热度衰减
    * -XX:CounterHalfLifeTime：设置半衰周期，单位是秒

****

# JVM体系结构
* 广义jvm：<img src='../images/javase/广义jvm.png' width='400px'>
* JVM体系结构结构图：<img src='../images/javase/jvm体系结构.png' width='400px'>
    * 类加载器，负责将.class中的字节码代码加载到JVM内存中
    * 执行引擎，负责解析字节码，执行字节码获得执行结果，每个执行引擎就是一个java线程
    * 内存管理
* JVM基于栈的架构：避免与操作系统平台产生关系，更好的优化性能，保证指令精简方便网络传输（备注其他的编译器一般是基于寄存器的架构）
* JVM执行引擎架构：<img src='../images/javase/jvm执行引擎结构.png' width='350px'>
    * JVM会为每个java线程生成一个栈（当成一个线性的数组结构），每个线程中调用的方法会创建一个栈帧，并且分配一个PC寄存器用于指定下条即将执行的指令
    * 基于栈架构的JVM任何操作都是结合栈的入栈和出栈操作完成计算，并且每次都会更新PC寄存器的值，保存下次即将操作的指令
    * 在调用方法时，首先是保存PC寄存器的值，然后创建新的栈帧，然后将PC寄存器值清0以便让它指向新栈栈的首行执行指令
    * 调用方法返回时，将方法结果返回给栈元素，并且将PC寄存器的值恢复到进入栈帧之前的值

****

# JVM内存管理
* JVM需要使用内存的部分：
    * java堆：存放对象的区域
    * 线程：每个线程都会分配一个栈空间，大概在256KB~756KB
    * 类和类加载器：也就是需要把JVM本身运行需要的类存放PermGen区中（永久代），参数-verbose:class查看JVM加载了那些类；如果PermGen不能对已失效的类做卸载，则会导致内存泄漏
    * NIO：可以直接映射系统内存进行IO操作，避免用户态到内核态的切换；NIO分配内存会导致一次os::malloc系统调用，分配的内存不是JVM内存是系统内存，因此也叫做堆外内存；正因为是堆外内存，所以释放过程只有系统GC才会触发，一般是手动调用系统GC清理，但是会造成性能下降GC次数变多
    * JNI：使得本地C语言方法可以调用Java方法
* JVM内存结构：
    * PC寄存器：负责存储下一个即将执行的指令，线程独有，且不共享
    * Java栈：为线程，本地变量（基本数据变量）、函数参数、对象引用提供存储空间；该区域不是线程共享的区域，其大小在javac编译时已经确定；主要负责执行程序，速度快效率高
    * 堆：为java对象提供存储空间，包括类实例和数组；该区域是线程共享该区域，其大小是动态分配的，由JVM负责管理其生命周期；主要负责存放对象，动态操作灵活性高
    * 方法区：存储类结构信息，包括常量池、字段、方法数据、方法体、构造函数、类中的专用方法、实例初始化、接口初始化；该区域存放在堆中的永久区中且可以被线程共享
    * 运行时常量池：跟方法区中的常量池是同一个东西，包括编译时期的数字常量、方法或字段的引用
    * 本地方法栈
    * 直接内存
* JVM内存回收，visualvm查看垃圾回收情况
    * JVM内存分配：
        * 静态内存分配（栈空间内存），在编译时期就确认大小，程序加载时一次性提供的内存，包括方法中的局部变量、基础变量、对象引用等；方法执行结束就释放
        * 动态内存分配（堆空间内存），对象创建时分配，对象没有被引用时释放，判断对象是否被引用：
            * 方法中局部变量区中对象的引用
            * java操作栈中的对象引用
            * 常量池中的对象引用，类静态属性引用
            * 本地方法中持有的对象引用
            * 类的Class对象
    * 垃圾回收算法（详情参考《深入理解java虚拟机》第77页）：
        * 算法主要思想：
            * 标记-清除算法：首先标记需要回收的对象，然后在一次性清除，效率低，标记过程要遍历；会产生大量空间碎片
            * 复制算法（Young区使用该算法思想）：将内存分为两块（Eden, Servivor），新建对象都存在Eden中，当Eden满了后将活的对象全部移到Servivor中，并清除Eden中的所有对象
            * 标记-清理算法（Old区使用该算法思想）：也是先标记活的对象，然后将他们移到内存的一端；然后将内存的另一端全部清除
            * 分代收集算法：下面的算法细节描述的就是分代算法如何分代了，其清理过程就是将复制算法和标记-清理算法整合应用到不同的区代中
        * 基于分代的垃圾收集算法（hotspot）：将堆分为几块（Young(Eden, Servivor)，Old，Permgen），新创建的对象放在年轻块，到经历过几次minor GC垃圾回收还没被回收的对象将会移到老年块中，减少对象扫描数量，提高效率；具体实现包括以下三种算法
        * Serial Collector：单线程回收，回收时会停止所有的线程工作
            * JVM在client模式下的默认垃圾回收算法
            * 新创建的对象都在Eden区，当超过PretenureSizeThreshold阈值或Eden区总大小后，会在Old区分配新对象
            * 当Eden区满了后，触发minor GC时会判断Old区剩余大小是否还可以容纳晋升的对象，如果不是则触发Full GC，如果是则判断HandlePromotionFailure的值（False触发）决定触发Full GC
        * Parallel Collector：分为ParNewGC、ParallelGC、ParallelOldGC
            * ParNewGC：机制与Serial Collector一致，是通过多线程回收
            * ParallelGC：JVM在server模式下的默认垃圾回收算法
            * ParallelOldGC：与ParallelGC几乎一样，区别在于Full GC时回收的内容不同
        * CMS(Concurrent Mark Sweep) Collector：
            * 通过判断Old区使用率来出发GC
        * 垃圾回收算法搭配使用图：<img src='../images/javase/回收算法结合使用.png' width='300px'>
* JVM GC日志分析，参考《JavaWeb深入分析技术内幕》第245页
* 内存交互协议以及限制：
    <table>
        <tr>
            <td><img src='../images/javase/内存交互1.png' width='350px'></td>
            <td><img src='../images/javase/内存交互2.png' width='350px'></td>
        </tr>
        <tr>
            <td><img src='../images/javase/交互限制1.png' width='350px'></td>
            <td><img src='../images/javase/交互限制2.png' width='350px'></td>
        </tr>
    </table>

****

# JVM解析步骤
* 编译过程：<img src='../images/javase/编译过程.png' width='400px'>
* Javac工作原理，将源码编译成字节码：（详情参考《深入理解java虚拟机》第279页）
    * 源代码示例：<img src='../images/javase/javac编译源示例代码.png' width='200px'>
    * 词法分析过程，读取源代码，定位关键字，找出规范化的Token流
        * Token流的解析顺序是通过java语言的语法顺序定义
        * 解析完的Token流：<img src='../images/javase/示例代码解析成token流.png' width='400px'>
    * 语法分析过程，解析Token，生成抽象语法树，语法检测发生在这步
        * 将Token流组件成结构化的语法树，也就是形成语句
        * 语法树的节点是XXXTree和JCTree的子类，最终每个节点是JCXXX类，每个类都表示一种特定的操作
        * 最终形成的语法树为：<img src='../images/javase/示例代码解析成语法树.png' width='400px'>
    * 注解处理器
        * 如果需要在编译时期加入注解处理器，需要```javac -processor 注解处理器类```，注解处理器类必须继承AbstractProcessor类，并实现process方法
        * 基础注解器操作参考《深入理解java虚拟机》第297页，区别注解符解析，有待进一步研究通过自己的框架
    * 语义分析过程，将难懂、复杂的语法转化为简单的语法
        * 优化语法树，包括添加默认构造函数、检查变量类型、检查操作语句是否可达、检查异常是否处理、解除语法糖等
        * 标注检查、数据及控制流分析、解释语法糖（语言特有的可省略的或者默认约定的语法，泛型、自动装箱、自动拆箱、遍历循环）
    * 最后将化简过的语法树通过字节码生成器生成字节码
        * 生成.class文件，将java代码会转化为Oolong汇编语言代码，详情在《JavaWeb深入分析技术内幕》145页
        * 可以通过javap生成比Oolong汇编语言代码更易懂的.class文件
* 即时编译器，运行时：（编译器优化等详情参考《深入理解java虚拟机》第308页）
    * 在javac编译完后，执行程序时，如果发现多次调用的方法或者多次执行的循环体会触发，将方法编译成本地方法
    * 执行时先是利用C0（解释器）执行
    * 其次当调用次数达到C1（client编译器）的阈值时，会通过C1将方法编译成本地方法，进行简单优化
    * 当调用次数达到C2（server编译器）的阈值时，会通过C2将方法编译成本地方法，进行深度优化
    * 触发垃圾回收是会触发热度衰减，也就是超过一定的时间限度（这段时间称为半衰周期），方法的调用次数没达到即时编译器的阈值，则会将方法调用器计数减半
    * 即时编译器介入的过程（方法计数器）：<img src='../images/javase/即时编译器介入.png' width='300px'>
    * 对于判断多次循环体，使用回边计数器，逻辑与方法计数器一致，回边计数器没有热度衰减
* class文件结构，16进制格式，详情参考《深入理解java虚拟机》第159页：
    * 魔数与class文件版本，前4字节时魔数表示该class文件是否能被当前jvm执行，5字节是主版本号，6字节是次版本号
    * 常量池，字面常量（String，Interge），符号引用（类和接口名字、字段名字和描述符、方法的名字和描述符）
    * 访问标志，如是类还是接口，是Public还是Private，多种标志会合并成一个16进制数
    * 类索引、父类索引、接口索引集合
    * 字段表集合，各标志出现顺序与字段定义时的顺序一致
    * 方法表集合，各标志出现顺序与方法定义时的顺序一致；方法返回值不参与重载，但是可以在class文件中作为区别唯一性
    * 属性表集合，用于存放类体，方法体的
* 类加载过程：
    * 加载：
        * 通过类的全限定名来获得此类的二进制字节流，应用实例：<img src='../images/javase/加载过程实例.png' width='300px'>
        * 将字节流代表的静态存储结构转化为方法区的运行时数据结构
        * 在java堆中生成一个代表该类的Class对象，作为方法区这些数据的访问入口
    * 验证：
        * 文件格式验证，验证class文件规范，是否可以被当前本部的虚拟机执行
        * 元数据验证，验证class文件中信息符合java语言规范要求，也就是验证相关的声明
        * 字节码验证，验证class文件中描述的数据流和控制流，也就是验证方法体的具体内容
        * 符号引用验证，验证访问权是否正确等信息
    * 准备：为类变量分配内存并且设置类变量的初始值（静态变量），全部都在方法区中
    * 解析：将常量池中的符号引用替换为直接引用，类或接口解析、字段解析、类方法解析、接口方法解析
    * 初始化：四种实例化才会触发类初始化，其他的都是被动引用
        * 遇到new、getstatic、putstatic、invokestatic四个字节指令（调用new、读取设置静态变量、调用类静态方法）
        * 通过反射对类进行反射调用
        * 初始化时触发父类初始化
        * 虚拟机启动时，指定的要执行的主类
* ClassLoader工作原理
    * 双亲委派模型：当类加载器收到加载请求时，会请求父加载器加载，当父类无法完成任务时才会自己处理
    * 主要任务：负责将.class文件加载到JVM内存中、检查哪个类该由谁加载、将.class文件字节码解析成JVM统一要求的对象格式
    * BootstrapClassLoader：加载JVM自身需要的类，完全由JVM控制，无父类和子类
    * ExtClassLoader：加载既不是JVM自身类，也不是应用程序类的其他类，主要是java.ext.dirs目录下的类
    * AppClassLoader：是ExtClassLoader子类，加载应用程序类，除了ExtClassLoader加载的类，所有的类都归该加载器加载
    * JVM加载类的方式：
        * 隐式加载：不通过在代码调用ClassLoader来加载需要的类，而是通过JVM来自动加载需要的类到内存的方式，如普通的new操作
        * 显示加载：在代码中调用ClassLoader来加载需要的类，如```this.getClass.getClassLoader().loadClass(); Class.forName();```
    * .class文件加载到JVM内存的步骤：<img src='../images/javase/class文件加载步骤.png' width='450px'>
        * 加载字节码到内存，主要是findClass()方法，BootstrapClassLoader查找class文件的参数为```-Xbootclasspath:, -Xbootclasspath/a:, -Xbootclasspath/p:```；ExtClassLoader查找class文件的参数为```-Djava.ext.dirs```;AppClassLoader查找class文件的参数为```-Djava.class.path=, -cp, -classpath```
        * 验证与解析：字节码验证；类准备，准备类所需要的字段，方法，接口等；解析，加载类引用的其他类
        * 初始化类对象，初始化静态字段
    * 类加载过程中的异常：ClassNotFoundException（显示加载找不到类），NoClassDefFoundError（隐式加载找不到类，new关键字，类中引用其他类、接口），UnsatisfiedLinkError（JVM自己lib被删除），ClassCastException（类型转换异常），ExceptionInInitializerError
    * 当类加载器加载完类后不记录类的状态，即用完就删除，就可以实现解释性语言    